In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from jproperties import Properties
import json
import pandas as pd

In [ ]:
configs = Properties()
with open('application.properties', 'rb') as config_file:
    configs.load(config_file)

In [ ]:
dotabuff_filters = configs.get("dotabuff_filters").data
dis_pkl_file_path = configs.get("json_file_path").data
wr_pkl_file_path = configs.get("json_file_path").data

In [ ]:
url1 = "https://www.dotabuff.com/heroes"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

req = requests.get(url1, headers=headers)
soup = BeautifulSoup(req.content, "html.parser")
hero_links = soup.find_all("a", href=re.compile(r"/heroes/.*"))
heroes=[]
for link in hero_links:
    if link.find("div", class_="hero"):
        heroes.append({"hero_name": link.text.strip(), "link_name": link['href'][8:]})

In [ ]:
all_hero_names_list = [hero_dict['hero_name'] for hero_dict in heroes]

print(json.dumps(all_hero_names_list))
with open("all_hero_names_list.json", "w") as outfile:
    outfile.write(json.dumps(all_hero_names_list))

In [ ]:
data_dict_dis={}
data_dict_wr={}
for hero in heroes:
    single_hero_counter_data=[]
    url2 = f"https://www.dotabuff.com/heroes/{hero['link_name']}/counters{dotabuff_filters}"
    req = requests.get(url2, headers=headers)
    soup = BeautifulSoup(req.content, "html.parser")
    header = soup.find("header", string="Matchups")
    table = header.find_next_sibling("article").find("table")
    for row in table.find_all("tr"):
        counter_name_tag = row.find("a", class_="link-type-hero")
        if row.find("td") and counter_name_tag:
            disadvantage = row.find_all("td")[2]['data-value']
            wr = row.find_all("td")[3]['data-value']
            data_dict_dis.setdefault(counter_name_tag.text.strip(), {})[hero['hero_name']] = float(disadvantage)
            data_dict_wr.setdefault(counter_name_tag.text.strip(), {})[hero['hero_name']] = float(wr)
    print(hero['hero_name'])

In [ ]:
df_dis = pd.DataFrame.from_dict(data_dict_dis, orient='index')
df_wr = pd.DataFrame.from_dict(data_dict_wr, orient='index')
df_dis = df_dis.fillna(0)
df_wr = df_wr.fillna(50)

In [ ]:
df_dis

In [ ]:
df_wr

In [ ]:
import pickle
with open('dbuff_dis_data.pkl', 'wb') as f:
    pickle.dump(df_dis, f)
with open('dbuff_wr_data.pkl', 'wb') as f:
    pickle.dump(df_wr, f)

In [ ]:
def sync_hero_names():
    url1 = "https://www.dotabuff.com/heroes"
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    req = requests.get(url1, headers=headers)
    soup = BeautifulSoup(req.content, "html.parser")
    hero_links = soup.find_all("a", href=re.compile(r"/heroes/.*"))
    heroes = []
    for link in hero_links:
        if link.find("div", class_="hero"):
            heroes.append({"hero_name": link.text.strip(), "link_name": link['href'][8:]})

    all_hero_names_list = [hero_dict['hero_name'] for hero_dict in heroes]
    with open('../assets/data/all_hero_names_list.json', 'w') as hero_names_out:
        hero_names_out.write(json.dumps(all_hero_names_list))
    return heroes

In [ ]:
heroes = sync_hero_names()

In [ ]:
heroes